<a href="https://www.kaggle.com/code/cocolabhub/10-02-generativellmwithhuggingface-datasets?scriptVersionId=221921721" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 데이터셋 관련\
- 실습 데이터(GTP 통해 생성) : https://huggingface.co/datasets/shangrilar/ko_text2sql
- 게임 데이터베이스 맞게 형성

In [1]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you ha

### 데이터 로딩
- 영어 text2sql : https://huggingface.co/datasets/Salesforce/wikisql
- 한글어 관련 데이터 hub : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71351

In [2]:
# !rm -rf *
# !ls -lah

In [3]:
from datasets import load_dataset
# https://huggingface.co/datasets/shangrilar/ko_text2sql/viewer/origin
hf_dataset_text2sql = load_dataset('shangrilar/ko_text2sql', 'origin')
hf_dataset_text2sql

Generating train split:   0%|          | 0/38246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 38246
    })
    test: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 112
    })
})

In [4]:
df_dataset_text2sql_train = hf_dataset_text2sql['train'].to_pandas()
df_dataset_text2sql_train = df_dataset_text2sql_train.dropna().sample(frac=1, random_state=42)

In [5]:
# huggingface dataset to pandas DataFrame with train
df_dataset_text2sql_train = df_dataset_text2sql_train.query('db_id != 1')  # 정상 데이터들만 추출
df_dataset_text2sql_train.head(3)

,db_id,context,question,answer
5857,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...
29093,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u..."
8040,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...


### 학습 위한 prompt 명령어 완성

In [6]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

In [7]:
# 필요한 컬럼 추출해 프롬프트 명령어 만드리
from tqdm import tqdm

df_total_count = len(df_dataset_text2sql_train)
for idx, row in tqdm(df_dataset_text2sql_train.iterrows(), total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
    # make_prompt(ddl, question, query='')
    train_promt_command = make_prompt(ddl= row['context']
                        , question = row['question']
                        , query=row['answer'])
    df_dataset_text2sql_train.loc[idx, 'text'] = train_promt_command
    pass

df_dataset_text2sql_train.head(3)

Generating prompts: 100%|██████████| 33876/33876 [00:06<00:00, 5425.02it/s]


,db_id,context,question,answer,text
5857,3,"CREATE TABLE categories (\n ""category_id"" S...",카테고리 이름을 길이가 긴 순서로 정렬하되 가장 긴 이름 3개만 보여줘,SELECT name FROM categories ORDER BY LENGTH(na...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
29093,4,"CREATE TABLE users (\n ""user_id"" SERIAL PRI...",카드로 결제한 사용자 중에 이름이 '김'씨씨인 사용자의 이메일과 카드 번호를 찾아주세요.,"SELECT u.email, pm.card_number FROM users AS u...",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
8040,4,"CREATE TABLE subscriptions (\n ""subscriptio...",시작일이 2022년 1월 1일 이후인 유효한 구독의 유저 아이디를 보여줘,SELECT DISTINCT user_id FROM subscriptions WHE...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [8]:
!mkdir ./datasets

### Test 위한 prompt 명령어 완성

In [9]:
# 재사용 위한 저장
# dataset_path = 'datasets'
# df_dataset_text2sql_train.to_csv(f'{dataset_path}/train.csv')

In [10]:
# huggingface dataset to pandas DataFrame with test
# df_dataset_text2sql_test = hf_dataset_text2sql['test'].to_pandas()
# df_dataset_text2sql_test.head(3)

In [11]:
# 필요한 컬럼 추출해 프롬프트 명령어 만드리
# from tqdm import tqdm

# df_total_count = len(df_dataset_text2sql_test)
# for idx, row in tqdm(df_dataset_text2sql_test.iterrows()
#                      , total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
#     # make_prompt(ddl, question, query='')
#     train_promt_command = make_prompt(ddl= row['context']
#                         , question = row['question']
#                         , query='')
#     df_dataset_text2sql_test.loc[idx, 'prompt'] = train_promt_command
#     pass

# df_dataset_text2sql_test.head(3)

In [12]:
# df_dataset_text2sql_test.to_csv(f'{dataset_path}/test.csv')

## linux 명령어 알기

In [13]:
!ls -la
# !ls -la ../
# !pwd

total 72
drwxr-xr-x 3 root root  4096 Feb 11 07:04 .
drwxr-xr-x 6 root root  4096 Feb 11 07:04 ..
drwxr-xr-x 2 root root  4096 Feb 11 07:04 datasets
---------- 1 root root 60592 Feb 11 07:04 __notebook__.ipynb


In [14]:
!zip -r GenerativeLLMwithHuggingface_Datasets.zip *

  adding: datasets/ (stored 0%)
  adding: __notebook__.ipynb (deflated 89%)
